Import Python libraries

In [ ]:
import numpy as np
import scipy.io
from sklearn.linear_model import ElasticNet
from __future__ import print_function
from sklearn.linear_model import BayesianRidge
from sklearn.feature_selection import VarianceThreshold
import sklearn

sklearn.__version__

Load Connectome data, convert into feature vectors and mComp2 scores.

In [3]:
#Loading Data, return X and Y

def load_data_noPreprocessing_MS2(normalize = False):
    m = scipy.io.loadmat("F.mat")
    m = scipy.io.loadmat("/data/eric/UNC/work/F.mat")

    y = []
    X = []

    for i in range(len(m['F'][0])):
        connectome = m['F'][0,i][0][0][22]
        group = m['F'][0,i][0][0][23]
        #change to 23 to return binary values of MS score. (21 for ranged values, 25 for 3bins)

        X.append( np.array( connectome )[0] )
        y.append( group[0,0] )
        
    if (normalize):
        #now, normalize the x data:
        print("normalizing data")
        for i in range(len(X)):
            maxX = max(X[i])
            for j in range(len(X[i])):
                X[i][j] = X[i][j] / maxX
        
        
    return X, y

Grid Search Function to find best possible parameters of computer model.

In [ ]:
#Grid search to find best alpha and l1ratio:

from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
kf = KFold(n_splits=10,shuffle=True)

X,y = load_data_noPreprocessing_MS2()


bestDifference = 100
bestCombo = ""

#Create the gridsearch of a's and l1's:
for a in range(20):
    allCVals = []
    for b in range(3):#rerun the same C val multiple times
        cval = a *3+1

        fold = 1
        #Kfold Loop:
        kfoldDiff = []
        kf = KFold(n_splits=10,shuffle=True)
        for train_index, test_index in kf.split(X):
            fold += 1

            #Create the test and training data:
            xTrain = []
            yTrain = []
            xTest = []
            yTest = []

            #Fill in the test and training data.
            for i in range(len(train_index)):
                xTrain.append(X[train_index[i]])
                yTrain.append(y[train_index[i]])

            for i in range(len(test_index)):
                xTest.append(X[test_index[i]])
                yTest.append(y[test_index[i]])   

            #Create the model based on the grid values for alpha and l1
            predictor = RandomForestRegressor(n_estimators=10, min_samples_split = 0.35)
            predictor.fit(xTrain, yTrain)

            diffs = []
            for i in range(len(xTest)):
                prediction = predictor.predict([xTest[i]])[0]
                diff = abs(yTest[i] - prediction)
                
                if prediction > yTest[i]:
                    diff = prediction - yTest[i]
                else: diff = yTest[i] - prediction
                
                diffs.append(diff)
                kfoldDiff.append(diff)

        print("Change value",cval,"run number:",b,"KFold diff:",np.mean(kfoldDiff))
        if (np.mean(kfoldDiff) < bestDifference):
            bestDifference = np.mean(kfoldDiff)
            bestCombo = "a:" + str(cval)
        allCVals.append(np.mean(kfoldDiff))
    print("Average diff:",np.mean(allCVals),"\n")
    
print("\n\nFinal Results:\nY value standard deviation:",np.std(y))
print("bestDiff:",bestDifference, "\n", "Best combo:", bestCombo)

Classification Model for MComp2 Scores

In [ ]:
##MCOMP2CLASSIFICATION

from sklearn.model_selection import KFold
from sklearn.neural_network import *
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import *
from sklearn.linear_model import LinearRegression
from sklearn import *
from sklearn.tree import *
import random
kf = KFold(n_splits=10,shuffle=True)

X,y = load_data_noPreprocessing_MS2()

def run(times,show):
    allAcc = []
    allRandAcc = []
    for t in range(times):
        fold = 1
        #Kfold Loop:

        kfoldDiff = []
        kfoldFixDiff = []
        histData = []
        coefSum = [0] * len(X[0])

        accuracy = []#only add in 1 for correct, 0 for incorrect.
        randAccuracy = []

        for train_index, test_index in kf.split(X):
            if show:print("Fold:",fold,"\nPred\tActual\tPred diff\tfixed Diff")
            fold += 1

            #Create the test and training data:
            xTrain = []
            yTrain = []
            xTest = []
            yTest = []

            #Fill in the test and training data.
            for i in range(len(train_index)):
                xTrain.append(X[train_index[i]])
                yTrain.append(y[train_index[i]])

            for i in range(len(test_index)):
                xTest.append(X[test_index[i]])
                yTest.append(y[test_index[i]])   


            #Define the model:
            ##predictor = ElasticNet(alpha=0.25, l1_ratio=0.35)
            #predictor = svm.SVC(C=0.8)
            ##predictor = ElasticNet(alpha=0.25, l1_ratio=0.45)
            ##predictor = svm.LinearSVC(C=0.9)
            ##predictor = svm.LinearSVC(C=0.01)s
            predictor = LinearRegression()
            ##predictor = ExtraTreeClassifier()
            ##predictor = RandomForestRegressor(n_estimators=10, min_samples_split = 0.35)
            ##predictor = Lasso(alpha=0.45)
            ####predictor = DecisionTreeRegressor(random_state=0)
            ##predictor = MLPClassifier(solver='lbfgs', alpha=0.7,
            ##                hidden_layer_sizes=(len(xTrain),len(xTrain),len(xTrain)), random_state=1)
            predictor.fit(xTrain, yTrain)


            diffs = []
            fixDiffs = []
            for i in range(len(xTest)):
                #Create a fixed pred that always guesses the average value of training Y.
                fixedPred = int(np.median(yTrain))
                fixedPred = random.choice(yTrain)

                prediction = round(predictor.predict([xTest[i]])[0])
                histData.append(yTest[i] - prediction)
                diff = abs(yTest[i] - prediction)

                if prediction > yTest[i]:
                    diff = prediction - yTest[i]
                else: diff = yTest[i] - prediction

                #Collect accuracy:
                if diff == 0:
                    accuracy.append(1)
                else: accuracy.append(0)

                if fixedPred > yTest[i]:
                    fixDiff = fixedPred - yTest[i]
                else: fixDiff = yTest[i] - fixedPred

                if fixedPred == yTest[i]:
                    randAccuracy.append(1)
                else: randAccuracy.append(0)

                if show: print(int(prediction),"\t",yTest[i], "\tDiff:",int(diff),"\t",(fixDiff))
                diffs.append(diff)
                kfoldDiff.append(diff)
                fixDiffs.append(fixDiff)
                kfoldFixDiff.append(fixDiff)

            if show:print("Accuracy:",1.0 - float(sum(diffs)) / float(len(diffs)))
            if show:print("Differences:",float(sum(diffs)) / float(len(diffs)))
        
        if show:print("KFold prediction Difference:",(np.mean(kfoldDiff)),"\tkfold random Difference:",(np.mean(kfoldFixDiff)))
        if show:print("Y value standard deviation:",np.std(y))

        if show:print("\n\nChance to guess correct bin:",np.mean(accuracy))
        if show:print("Random chance to guess correct bin:",np.mean(randAccuracy))
        allAcc.append(np.mean(accuracy))
        allRandAcc.append(np.mean(randAccuracy))
        print(t,",",end="")
    print("\nAll Accuracy:",np.mean(allAcc))
    print("Rand Accuracy:",np.mean(allRandAcc))
    
run(200,False)

Predictive model for Mcomp2 Scores

In [ ]:
##MCOMP2 SCORE PREDICTIONS
from sklearn.neural_network import *
from random import *
from sklearn.model_selection import KFold
from scipy import stats

def getData(normalize = False):
    m = scipy.io.loadmat("F.mat")
    m = scipy.io.loadmat("/data/eric/UNC/work/F.mat")

    y = []
    X = []
    groups = []
    for i in range(len(m['F'][0])):
        connectome = m['F'][0,i][0][0][22]
        group = m['F'][0,i][0][0][21]
        typeGroup = m['F'][0,i][0][0][6][0]
        #change to 23 to return binary values of MS score. (21 for ranged values, 25 for 3bins)

        X.append( np.array( connectome )[0] )
        y.append( group[0,0] )
        groups.append(typeGroup)
        
    if (normalize):
        #now, normalize the x data:
        print("normalizing data")
        for i in range(len(X)):
            maxX = max(X[i])
            for j in range(len(X[i])):
                X[i][j] = X[i][j] / maxX
        
        
    return X, y, groups
   

#Execute this function
def run(algo,show=True,updatedFold=False):
    X,y,groups = getData()
    kf = KFold(n_splits=10,shuffle=True)
    
    fold = 0
    
    singletonDiffs = []
    twinDiffs = []
    riskDiffs = []
    allDiffs = []
    
    for train_index, test_index in kf.split(X):
            fold += 1
            if show:print("\nFold:",fold,"\nActual\tPredicted\tDifference\tGroup")
            
            #Create the test and training data:
            xTrain = []
            yTrain = []
            xTest = []
            yTest = []
            groupTrain = []
            groupTest = []
        
            if updatedFold:#fill in training data with updated algo.
                testIndexes = updatedIndexPicker()
                thisTestIndexes = testIndexes[fold-1]
                
                #Iterate through X.  If the current index is in the fold's
                #test values, add that X index to xTest.  Same for Y and group.
                for i in range(len(X)):
                    if i in thisTestIndexes:
                        xTest.append(X[i])
                        yTest.append(y[i])   
                        groupTest.append(groups[i])
                    else:
                        xTrain.append(X[i])
                        yTrain.append(y[i])
                        groupTrain.append(groups[i])
                shuffle(yTrain)
            else:
                #Fill in the test and training data.
                for i in range(len(train_index)):
                    xTrain.append(X[train_index[i]])
                    yTrain.append(y[train_index[i]])
                    groupTrain.append(groups[train_index[i]])

                for i in range(len(test_index)):
                    xTest.append(X[test_index[i]])
                    yTest.append(y[test_index[i]])   
                    groupTest.append(groups[test_index[i]])
                
                
            #Define the model:
            algos = [ElasticNet(alpha=0.25, l1_ratio=0.45),
                     LinearRegression(),
                     RandomForestRegressor(n_estimators=10, min_samples_split = 0.35),
                     Lasso(alpha=0.45),
                     DecisionTreeRegressor(random_state=0)]
            predictor = algos[algo]
            
            predictor.fit(xTrain, yTrain)


            diffs = []
            #make predictions for every test index
            for i in range(len(xTest)):
                #Create a fixed pred that always guesses the average value of training Y.
                prediction = (predictor.predict([xTest[i]])[0])
                diff = abs(yTest[i] - prediction)
                if show:print(yTest[i],"\t","%0.3f"%prediction,"\t","%0.2f"%diff,"\t\t",groupTest[i])
                #print(groupTest[i][0])
                
                #Now, add the differences to each of the groups:
                if (groupTest[i][0] == "S"):
                    singletonDiffs.append(diff)
                elif (groupTest[i][0] == "T"):
                    twinDiffs.append(diff)
                else:
                    riskDiffs.append(diff)
                allDiffs.append(diff)
                
    if show:print("\nSingleton difference:",np.mean(singletonDiffs))
    if show:print("Twin difference:",np.mean(twinDiffs))
    if show:print("High Risk difference:",np.mean(riskDiffs))
    if show:print("Average difference:",np.mean(allDiffs))
    if show:print("Standard dev:",np.std(allDiffs))#about 7-9
    
    return np.mean(singletonDiffs),np.mean(twinDiffs),np.mean(riskDiffs),np.mean(allDiffs)


def runTimes(algo,times):
    allSingDiffs = []
    allTwinDiffs = []
    allRiskDiffs = []
    allAllDiffs = []
    
    for i in range(times):
        print(".",end="",sep="")
        sing,twin,risk,allD = run(algo,show=False,updatedFold=True)
        allSingDiffs.append(sing)
        allTwinDiffs.append(twin)
        allRiskDiffs.append(risk)
        allAllDiffs.append(allD)
    print("\n\nAverage Singleton Difference:",np.mean(allSingDiffs))
    print("Average Twin Difference:",np.mean(allTwinDiffs))
    print("Average High Risk Difference:",np.mean(allRiskDiffs))
    print("Average All Difference:",np.mean(allAllDiffs))


#Run all of the algorithms
print("Elastic net:")
runTimes(0,30)

print("\n\nLinear Regression:")
runTimes(1,30)

print("\n\nRandom Forest Regressor:")
runTimes(2,30)

print("\n\nLasso:")
runTimes(3,50)

print("\n\nDecision Tree Regressor:")
runTimes(4,30)
#"""

Draw the neural network data.  First, read the output file and parse into usable data.

In [2]:
from scipy import io
from numpy.linalg import norm
import numpy as np
from __future__ import print_function

def load_data_noPreprocessing_MS2(lay1Len,lay2Len,lay3Len,lay4Len):
    m = io.loadmat("T.mat")

    y = []
    X = []

    layers = [[],[],[],[],[]]
    
    #set up the layer combos
    for i in range(lay1Len):
        layers[0].append([1000])
    for i in range(lay2Len):
        layers[1].append([2000])
    for i in range(lay3Len):
        layers[2].append([3000])
    for i in range(lay4Len):
        layers[3].append([4000])
    for i in range(300):
        layers[4].append([5000])
    
    
    for i in range(len(m['T'][0])):
        theseInodes = m['T'][0][i][0][0][2]
        position = m['T'][0][i][0][0][1][0][0]
        thisLayer = m['T'][0][i][0][0][0][0][0]
        newInodes = []
        for j in range(len(theseInodes)):
            newInodes.append(theseInodes[j][0])

        layers[thisLayer-1][position-1] = newInodes

    return layers
    
layers = load_data_noPreprocessing_MS2(4,6,11,50)   


allPaths = []
counter = 1
def getChildren(path,index,currentLayer,layers):
    if (index>= len(layers[currentLayer])): return
    inodes = layers[currentLayer][index]
    
    if currentLayer == 4  and max(inodes) < 500:
        paths = []
        for i in range(len(inodes)):
            thisPath = path[:]
            thisPath.append(inodes[i])
            
            allPaths.append(thisPath)
            print("\t\t\t\tPATH",thisPath)
    #otherwise, continue to the next layer:
    else:
        if currentLayer == 0 and max(inodes) < 500:
            print("firstlayer:",index+1,inodes)
        if currentLayer == 1 and max(inodes) < 500:
            print("\tsecondlayer:",index+1,inodes)
        if currentLayer == 2 and max(inodes) < 500:
            print("\t\tthirdlayer:",index+1,inodes)
        if currentLayer == 3 and max(inodes) < 500:
            print("\t\t\tfourthlayer:",index+1,inodes)
        for i in range(len(inodes)):
            if inodes[i] < 500:
                newPath = path[:]
                newPath.append(inodes[i])
                getChildren(newPath,inodes[i]-1,currentLayer +1,layers)
 
getChildren([1],0,0,layers)
getChildren([2],1,0,layers)
getChildren([3],2,0,layers)
getChildren([4],3,0,layers)

for i in range(len(allPaths)):
    print("Path",i,allPaths[i])


firstlayer: 1 [1, 2, 3, 4, 5, 6]
	secondlayer: 1 [2, 6, 10, 11]
		thirdlayer: 2 [11, 25, 39, 42]
		thirdlayer: 6 [3, 8, 28, 30, 37, 40, 44]
			fourthlayer: 3 [56, 95]
				PATH [1, 1, 6, 3, 56, 26]
				PATH [1, 1, 6, 3, 56, 102]
				PATH [1, 1, 6, 3, 95, 68]
				PATH [1, 1, 6, 3, 95, 130]
				PATH [1, 1, 6, 3, 95, 141]
			fourthlayer: 8 [6, 54]
				PATH [1, 1, 6, 8, 54, 22]
				PATH [1, 1, 6, 8, 54, 46]
				PATH [1, 1, 6, 8, 54, 59]
				PATH [1, 1, 6, 8, 54, 82]
			fourthlayer: 28 [84, 86]
				PATH [1, 1, 6, 28, 84, 115]
				PATH [1, 1, 6, 28, 84, 175]
				PATH [1, 1, 6, 28, 86, 102]
				PATH [1, 1, 6, 28, 86, 106]
		thirdlayer: 10 [2, 3, 10, 19, 20, 29, 31, 34]
			fourthlayer: 3 [56, 95]
				PATH [1, 1, 10, 3, 56, 26]
				PATH [1, 1, 10, 3, 56, 102]
				PATH [1, 1, 10, 3, 95, 68]
				PATH [1, 1, 10, 3, 95, 130]
				PATH [1, 1, 10, 3, 95, 141]
			fourthlayer: 10 [41, 49]
				PATH [1, 1, 10, 10, 49, 28]
				PATH [1, 1, 10, 10, 49, 61]
				PATH [1, 1, 10, 10, 49, 159]
			fourthlayer: 20 [

Path 103 [2, 1, 10, 10, 49, 159]
Path 104 [2, 1, 10, 20, 56, 26]
Path 105 [2, 1, 10, 20, 56, 102]
Path 106 [2, 1, 11, 8, 54, 22]
Path 107 [2, 1, 11, 8, 54, 46]
Path 108 [2, 1, 11, 8, 54, 59]
Path 109 [2, 1, 11, 8, 54, 82]
Path 110 [2, 2, 1, 45, 38, 19]
Path 111 [2, 2, 1, 45, 38, 80]
Path 112 [2, 2, 4, 3, 56, 26]
Path 113 [2, 2, 4, 3, 56, 102]
Path 114 [2, 2, 4, 3, 95, 68]
Path 115 [2, 2, 4, 3, 95, 130]
Path 116 [2, 2, 4, 3, 95, 141]
Path 117 [2, 2, 11, 8, 54, 22]
Path 118 [2, 2, 11, 8, 54, 46]
Path 119 [2, 2, 11, 8, 54, 59]
Path 120 [2, 2, 11, 8, 54, 82]
Path 121 [2, 4, 3, 3, 56, 26]
Path 122 [2, 4, 3, 3, 56, 102]
Path 123 [2, 4, 3, 3, 95, 68]
Path 124 [2, 4, 3, 3, 95, 130]
Path 125 [2, 4, 3, 3, 95, 141]
Path 126 [2, 4, 3, 24, 20, 94]
Path 127 [2, 4, 3, 24, 20, 174]
Path 128 [2, 4, 3, 24, 20, 177]
Path 129 [2, 4, 3, 24, 39, 97]
Path 130 [2, 4, 3, 24, 39, 98]
Path 131 [2, 4, 3, 24, 39, 179]
Path 132 [2, 4, 6, 3, 56, 26]
Path 133 [2, 4, 6, 3, 56, 102]
Path 134 [2, 4, 6, 3, 95, 68]
Path 1

Now, use the parsed data to draw to screen

In [1]:
#https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html


import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
     
def main():
    
    connections = []
    for i in range(len(allPaths)):
        #Add in all of the nodes:
        connections.append((""+ str(allPaths[i][0]) + ""," "+str(allPaths[i][1])+ " " ))
        connections.append((" "+ str(allPaths[i][1]) + " ","  "+str(allPaths[i][2])+ "  " ))
        connections.append(("  "+ str(allPaths[i][2]) + "  ","   "+str(allPaths[i][3])+ "   " ))
        connections.append(("   "+ str(allPaths[i][3]) + "   ","    "+str(allPaths[i][4])+ "    " ))
        connections.append(("    "+ str(allPaths[i][4]) + "    ","     "+str(allPaths[i][5])+ "     " ))
        
    # Create a directed graph
    G = nx.DiGraph()
    l = connections
     
    # Build up a graph
    for t in l:
        G.add_edge(t[0], t[1])
     
    # Plot trees
    pos=graphviz_layout(G, prog='dot')
    nx.draw_networkx(G, pos, with_labels=True, arrows=False,node_size=350,node_shape='s',node_color="#aef2c6")
    plt.savefig('draw_trees_with_pygraphviz.png', bbox_inches='tight')   
    plt.show()
     
if __name__ == '__main__':
    main()

NameError: global name 'allPaths' is not defined